# Carga de Datos

## Importación de librerías

In [1]:
import os.path
import pandas as pd
import sqlite3

## Constantes

Rutas usuales que se ocuparán en el notebook

In [2]:
# Salvar DataFrame?
save_df = False

# RUTAS
DATA_PATH = os.path.join("..", "data")
WF_FOLDER_PATH = os.path.join(DATA_PATH, "wildfires_us")
WF_DATA_PATH = os.path.join(WF_FOLDER_PATH, "FPA_FOD_20170508.sqlite")
WF_DATA_COLUMNS_PATH = os.path.join(WF_FOLDER_PATH, "COLUMNS")

## Funciones auxiliares

Función que se ocupará en la carga de datos.

In [3]:
def crear_puntero(path=WF_DATA_PATH, msg=False):
    """Función que crea un puntero con el dataset de incendios forestales en US.
    """
    try:
        conn = sqlite3.connect(path)
        if msg:
            print("Conexión realizada con éxito.")
    except:
        if msg:
            print("No se encuentra el archivo.")
    return conn

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [4]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

Función que se ocupará para crear una columna extra en el dataset

In [5]:
def generar_datetime(fecha_col, hora_col):
    """Recibe un DataFrame 'df', una columna 'fecha_col' que tiene 
    la fecha en formato de marca de tiempo (Timestamp) y una columna 
    'hora_col' que tiene la hora en formato 'hhmm'.

    Retorna una columna del tipo 'Series', de datos del tipo datetime,
    con la fecha y hora juntas.
    """

    # Transformamos la columna de fechas a la fecha "usual"
    fecha_col_ = pd.to_datetime(fecha_col - pd.Timestamp(0).to_julian_date(),
                                unit='D',
                                dayfirst=True)
    
    # Obtenemos solo la parte de la fecha como una columna de str
    fecha_col_ = fecha_col_.map(lambda x: str(x).split()[0])

    # Procesamos la columna de la hora para que esté en formato "hh:mm:ss"
    hora_col_ = hora_col.map(lambda x: ":".join([" " + x[0:2], x[2:4], "00"]))

    # Retornamos la suma de ambas, en formato de datetime
    return pd.to_datetime(fecha_col_ + hora_col_)

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [6]:
# Todas las columnas
columnas = list(pd.read_csv(WF_DATA_COLUMNS_PATH))

# Columnas que se ocuparán en el análisis
columnas_ocupadas = [
    "FIRE_YEAR",        # Año en que el incendio fue descubierto o confirmado
    "DISCOVERY_DATE",   # Fecha en que fue descubiertoel incendio
    "DISCOVERY_TIME",   # Hora en que fue descubierto el incendio
    "STAT_CAUSE_DESCR", # Descripción de la causa del incendio
    "CONT_DATE",        # Fecha en que se contuvo el incendio
    "CONT_TIME",        # Hora en que se contuvo el incendio
    "FIRE_SIZE",        # Área final estimada que alcanzó el incendio
    "FIRE_SIZE_CLASS",
    "LATITUDE",         # Latitud de la localización del incendio
    "LONGITUDE",        # Longitud de la localización del incendio
    "STATE",
    "COUNTY",
    "FIPS_NAME",
]

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [7]:
# str que se ocupará al cargar los datos
columnas_str = ",".join(columnas_ocupadas)

# Obtener una fila del dataset
conn = crear_puntero(msg=True)
df = pd.read_sql_query(f"SELECT {columnas_str} FROM 'Fires'", conn)
conn.close()

del columnas_str, conn

print_cantidad(df)

df.head()

Conexión realizada con éxito.
El dataset tiene una cantidad de 1880465 datos y 13 variables.


,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_TIME,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_NAME
0,2005,2453403.5,1300,Miscellaneous,2453403.5,1730,0.10,A,40.036944,-121.005833,CA,63,Plumas
1,2004,2453137.5,0845,Lightning,2453137.5,1530,0.25,A,38.933056,-120.404444,CA,61,Placer
2,2004,2453156.5,1921,Debris Burning,2453156.5,2024,0.10,A,38.984167,-120.735556,CA,17,El Dorado
3,2004,2453184.5,1600,Lightning,2453189.5,1400,0.10,A,38.559167,-119.913333,CA,3,Alpine
4,2004,2453184.5,1600,Lightning,2453189.5,1200,0.10,A,38.559167,-119.933056,CA,3,Alpine


## Descripción preliminar de los datos

### Manejo de valores nulos

Observemos la información actual del df

In [8]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 13 columns):
 #   Column            Dtype  
---  ------            -----  
 0   FIRE_YEAR         int64  
 1   DISCOVERY_DATE    float64
 2   DISCOVERY_TIME    object 
 3   STAT_CAUSE_DESCR  object 
 4   CONT_DATE         float64
 5   CONT_TIME         object 
 6   FIRE_SIZE         float64
 7   FIRE_SIZE_CLASS   object 
 8   LATITUDE          float64
 9   LONGITUDE         float64
 10  STATE             object 
 11  COUNTY            object 
 12  FIPS_NAME         object 
dtypes: float64(5), int64(1), object(7)
memory usage: 186.5+ MB


Primero observamos el número de elementos nulos que tiene el dataset.

In [9]:
df.isna().sum()

FIRE_YEAR                0
DISCOVERY_DATE           0
DISCOVERY_TIME      882638
STAT_CAUSE_DESCR         0
CONT_DATE           891531
CONT_TIME           972173
FIRE_SIZE                0
FIRE_SIZE_CLASS          0
LATITUDE                 0
LONGITUDE                0
STATE                    0
COUNTY              678148
FIPS_NAME           678148
dtype: int64

Dado al gran tamaño de la base de datos, nos podemos dar "el lujo" de eliminar las filas que contengan elementos nulos.

In [10]:
# Se quitan los valores nulos
df.dropna(inplace=True)

# Se resetean los índices
df.reset_index(drop=True, inplace=True)

# Se imprime la cantidad
print_cantidad(df)

# Y se muestran la cantidad de nulos que tiene actualmente
df.isna().sum()

El dataset tiene una cantidad de 581159 datos y 13 variables.


FIRE_YEAR           0
DISCOVERY_DATE      0
DISCOVERY_TIME      0
STAT_CAUSE_DESCR    0
CONT_DATE           0
CONT_TIME           0
FIRE_SIZE           0
FIRE_SIZE_CLASS     0
LATITUDE            0
LONGITUDE           0
STATE               0
COUNTY              0
FIPS_NAME           0
dtype: int64

Ahora revisamos el tipo de datos que tiene el Data Frame

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581159 entries, 0 to 581158
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   FIRE_YEAR         581159 non-null  int64  
 1   DISCOVERY_DATE    581159 non-null  float64
 2   DISCOVERY_TIME    581159 non-null  object 
 3   STAT_CAUSE_DESCR  581159 non-null  object 
 4   CONT_DATE         581159 non-null  float64
 5   CONT_TIME         581159 non-null  object 
 6   FIRE_SIZE         581159 non-null  float64
 7   FIRE_SIZE_CLASS   581159 non-null  object 
 8   LATITUDE          581159 non-null  float64
 9   LONGITUDE         581159 non-null  float64
 10  STATE             581159 non-null  object 
 11  COUNTY            581159 non-null  object 
 12  FIPS_NAME         581159 non-null  object 
dtypes: float64(5), int64(1), object(7)
memory usage: 57.6+ MB


### Manejo de las variables de tiempo

Notemos que `{DISCOVERY,CONT}_DATE` está en formato timestamp, y que `{DISCOVERY,CONT}_TIME` está en formato `'hhmm'`:

In [12]:
cols_aux = ["DISCOVERY_DATE", "DISCOVERY_TIME", 
            "CONT_DATE", "CONT_TIME"]

df[cols_aux].head()

,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME
0,2453403.5,1300,2453403.5,1730
1,2453137.5,0845,2453137.5,1530
2,2453156.5,1921,2453156.5,2024
3,2453184.5,1600,2453189.5,1400
4,2453184.5,1600,2453189.5,1200


Generamos una nueva columna que tenga la fecha y hora contenida. Pues la fecha está en formato Timestamp y la hora está en formato `'hhmm'`. Lo hacemos para los prefijos `'DISCOVERY_'` y `'CONT_'`.

In [13]:
# Creamos nuevas columnas
df["DISCOVERY_DATE_TIME"] = generar_datetime(df["DISCOVERY_DATE"],
                                             df["DISCOVERY_TIME"])
df["CONT_DATE_TIME"] = generar_datetime(df["CONT_DATE"],
                                        df["CONT_TIME"])

# Agregamos estas nuevas columnas al conjunto de columnas
nuevas_cols = ["DISCOVERY_DATE_TIME", "CONT_DATE_TIME"]
columnas += nuevas_cols
columnas_ocupadas += nuevas_cols

df[nuevas_cols].head()

,DISCOVERY_DATE_TIME,CONT_DATE_TIME
0,2005-02-02 13:00:00,2005-02-02 17:30:00
1,2004-05-12 08:45:00,2004-05-12 15:30:00
2,2004-05-31 19:21:00,2004-05-31 20:24:00
3,2004-06-28 16:00:00,2004-07-03 14:00:00
4,2004-06-28 16:00:00,2004-07-03 12:00:00


Quitamos ahora las columnas que resultan redundantes, al tener la misma información de los que tiene prefijo `_DATE_TIME`: 
* `DISCOVERY_DATE`, `DISCOVERY_TIME`.
* `CONT_DATE`, `CONT_TIME`.

In [14]:
for x in cols_aux:
    columnas.remove(x)
    columnas_ocupadas.remove(x)
    columnas_sin_ocupar.append(x)

del cols_aux, nuevas_cols
    
df = df[columnas_ocupadas]
print_cantidad(df)
df.head()

El dataset tiene una cantidad de 581159 datos y 11 variables.


,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_NAME,DISCOVERY_DATE_TIME,CONT_DATE_TIME
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,63,Plumas,2005-02-02 13:00:00,2005-02-02 17:30:00
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,61,Placer,2004-05-12 08:45:00,2004-05-12 15:30:00
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,17,El Dorado,2004-05-31 19:21:00,2004-05-31 20:24:00
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,3,Alpine,2004-06-28 16:00:00,2004-07-03 14:00:00
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,3,Alpine,2004-06-28 16:00:00,2004-07-03 12:00:00


Crearemos una columna de los meses y semanas, para ver si nos entrega información extra.

In [15]:
# Creamos las columnas {DISCOVERY,CONT}_{MONTH,DAY_OF_WEEK}
df["DISCOVERY_MONTH"] = df["DISCOVERY_DATE_TIME"].dt.month
df["DISCOVERY_DAY_OF_WEEK"] = df["DISCOVERY_DATE_TIME"].dt.day_name()

df["CONT_MONTH"] = df["CONT_DATE_TIME"].dt.month
df["CONT_DAY_OF_WEEK"] = df["CONT_DATE_TIME"].dt.day_name()

# Añadimos las nuevas columnas al conjunto de columnas
cdt = columnas_ocupadas.pop(-1)
columnas.pop(-1)
cols_aux = ["DISCOVERY_MONTH", "DISCOVERY_DAY_OF_WEEK", 
            cdt, "CONT_MONTH", "CONT_DAY_OF_WEEK"]

columnas.extend(cols_aux)
columnas_ocupadas.extend(cols_aux)
del cdt, cols_aux

# Actualizamos el df
df = df[columnas_ocupadas]

df.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_NAME,DISCOVERY_DATE_TIME,DISCOVERY_MONTH,DISCOVERY_DAY_OF_WEEK,CONT_DATE_TIME,CONT_MONTH,CONT_DAY_OF_WEEK
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,63,Plumas,2005-02-02 13:00:00,2,Wednesday,2005-02-02 17:30:00,2,Wednesday
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,61,Placer,2004-05-12 08:45:00,5,Wednesday,2004-05-12 15:30:00,5,Wednesday
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,17,El Dorado,2004-05-31 19:21:00,5,Monday,2004-05-31 20:24:00,5,Monday
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,3,Alpine,2004-06-28 16:00:00,6,Monday,2004-07-03 14:00:00,7,Saturday
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,3,Alpine,2004-06-28 16:00:00,6,Monday,2004-07-03 12:00:00,7,Saturday


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581159 entries, 0 to 581158
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   FIRE_YEAR              581159 non-null  int64         
 1   STAT_CAUSE_DESCR       581159 non-null  object        
 2   FIRE_SIZE              581159 non-null  float64       
 3   FIRE_SIZE_CLASS        581159 non-null  object        
 4   LATITUDE               581159 non-null  float64       
 5   LONGITUDE              581159 non-null  float64       
 6   STATE                  581159 non-null  object        
 7   COUNTY                 581159 non-null  object        
 8   FIPS_NAME              581159 non-null  object        
 9   DISCOVERY_DATE_TIME    581159 non-null  datetime64[ns]
 10  DISCOVERY_MONTH        581159 non-null  int64         
 11  DISCOVERY_DAY_OF_WEEK  581159 non-null  object        
 12  CONT_DATE_TIME         581159 non-null  date

## Salvar DataFrame

Salvamos el DataFrame en la carpeta respectiva.

In [17]:
if save_df:
    df.to_csv(os.path.join(WF_FOLDER_PATH, "WILDFIRES_USA.csv"), index=False)